In [2]:
import lancedb
import pyarrow as pa
import pandas as pd
from datasets import load_dataset
from transformers import GPT2Tokenizer

Para este ejercicio descargamos una base de datos de mensajes de texto en inglés.

In [3]:
dataset = load_dataset("chirunder/text_messages")

Generating train split: 100%|██████████| 11615290/11615290 [00:21<00:00, 549933.03 examples/s] 


In [5]:
df = pd.DataFrame(dataset['train'])
df.rename(columns={'text': 'texto'}, inplace=True)
df.head()

,texto
0,Top right I gained a little speed with the add...
1,They are heavier wheels though as are all the ...
2,Federally registering a trademark is more than...
3,I'll have to jog my memory from rooting a few ...
4,Unless you can afford to buy all new larger cl...


Para los siguientes ejercicios, voy a crear una variable del numero de palabras en cada mensaje de texto.

In [9]:
df['n'] = df['texto'].apply(lambda x: len(str(x).split()))
df = df[['n', 'texto']]
df.head()

,n,texto
0,13,Top right I gained a little speed with the add...
1,14,They are heavier wheels though as are all the ...
2,9,Federally registering a trademark is more than...
3,21,I'll have to jog my memory from rooting a few ...
4,10,Unless you can afford to buy all new larger cl...


## Task 1:
A partir del dataframe df, crea df_tokenized (usando el Tokenizer de GPT2) con dos columnas pero con el texto tokenizado. Asegurate de que todos los embeddings tengan la misma longitud y los tokens sean enteros (todos enteros o todos doubles). 

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

mensajes_tokenizados = df['texto'].apply(lambda x: tokenizer(x)["input_ids"])

mensajes_tokenizados = mensajes_tokenizados.apply(lambda x: x[:200] + [0] * (200 - len(x)) if len(x) < 200 else x[:200])

df_tokenized = pd.DataFrame({'vector': mensajes_tokenizados, 'name': df['n']})

## Task 2:
Mete el dataframe a una tabla en una base de datos de LanceDB.

In [ ]:
db = lancedb.connect("./mensajes.lancedb")
db.create_table("data", df_tokenized)
db["data"].head()

## Task 3:
Haz una query estilo SQL a la tabla de la base de datos. Quiero que escribas la query equivalente y pongas la explicación de lo que está haciendo la consulta. Hint: usa la columna "n". 

In [ ]:
(db["data"].search()
    .where("texto LIKE '%going to take a nap%'")
    .select(["name", "vector"])
    .limit(10)
    .to_pandas())

- Query en SQL equivalente:
- Explicacion: Busacos los 10 primeros mensajes que tengan el "me voy a e

## Task 4:
Inventa un mensaje de texto que tu podrías escribirle a un amigo. Tokenizalo y ponlo en el formato adecuado para hacer un vector query. Quiero que me regreses el mensaje más parecido al mensaje que inventaste (OJO: quiero el texto, no el embedding). HINT: Hay que decodear el resultado del query.

In [10]:
mensaje = 'Floppy has been the best teacher of my career'
mensaje = tokenizer(mensaje)["input_ids"]

menaje = mensaje.apply(lambda x: x[:200] + [0] * (200 - len(x)) if len(x) < 200 else x[:200])

query = (db["data"].search(mensaje)
        .metric("cosine")
        .select(["vector"])
        .limit(5)
        .to_pandas())

query_tokenizado = querry['vector'].values[0]

mensaje_parecido =  tokenizer.decode(query_tokenizado, skip_special_tokens=True)